In [1]:
import numpy as np
import ctypes
import pyautogui

from PIL import Image, ImageDraw
import decimal

from pytesseract import pytesseract
import pandas as pd
from tqdm import tqdm
import time

In [2]:
user32 = ctypes.windll.user32
screenSize = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)
centerPoint = tuple(i/2 for i in screenSize)
monitor_width = screenSize[0]
monitor_height = screenSize[1]

In [37]:
def drange(x, y, jump):
    while x < y:
        yield float(x)
        x += decimal.Decimal(jump)
        
def cell_dims(x1, y1, x2, y2):
    return x1, y1, x2 - x1, y2 - y1

def cell_dims_from_list(list_onbj):
    return cell_dims(list_onbj[0], list_onbj[1], list_onbj[2], list_onbj[3])

In [38]:
full_screen_image = pyautogui.screenshot(region=(0, 0, monitor_width, monitor_height))
survey_scan_results_box = [774, 1076, 967, 1100] #x,y Top Left ,x,y Bottom Right
crop = full_screen_image.crop(survey_scan_results_box) 
crop

In [39]:
im = crop.convert('RGBA')
data = np.array(im)
red, green, blue, alpha = data.T

# find and replace (cleaning up image)
white_areas = (red == 58) & (blue == 102) & (green == 92)
data[..., :-1][white_areas.T] = (14, 14, 14)
white_areas = (red == 35) & (blue == 62) & (green == 56)
data[..., :-1][white_areas.T] = (14, 14, 14)
white_areas = (red == 45) & (blue == 78) & (green == 71)
data[..., :-1][white_areas.T] = (14, 14, 14)

im2 = Image.fromarray(data)
im2

In [40]:
Processed_string = pytesseract.image_to_string(im2, lang='eng').replace('\n', '').replace(' m*', '').replace(',', '')
Processed_string.split('/')

['83398.8', '1811882.8']

In [4]:
def process():
    full_screen_image = pyautogui.screenshot(region=(0, 0, monitor_width, monitor_height))
    survey_scan_results_box = [1610, 1076, 2050, 1332] #x,y Top Left ,x,y Bottom Right
    
    space = (survey_scan_results_box[3] - survey_scan_results_box[1]) / 12
    row_points= list(drange(survey_scan_results_box[1], survey_scan_results_box[3], f'{space}'))
    final_row_points = list(np.rint(np.asarray(row_points)).astype(int))
    
    col_delta = survey_scan_results_box[2] - survey_scan_results_box[0]
    cols_points =  [col_delta * .05, col_delta * .45, col_delta * .15, col_delta * .18]

    temp_list = []

    for i in range(len(cols_points)):
        this_int = survey_scan_results_box[0]
        for j in range(i + 1):
            this_int += cols_points[j]
        temp_list.append(this_int)

    final_col_points = np.array(temp_list)

    final_col_points = list(np.rint(final_col_points).astype(int))
    final_col_points = [survey_scan_results_box[0]] + final_col_points + [survey_scan_results_box[2]]
    
    x_range = final_col_points
    y_range = final_row_points

    cells = {}

    for i, x in enumerate(x_range):
        for j, y in enumerate(y_range):
            try:
                cells[i, j] = tuple([x, y, x + x_range[i+1], y + y_range[j+1]])
            except:
                pass
            
    frames = []
    columns = ['Locked', 'Ore', 'Quantity', 'Volume', 'Distance', 'click_target']
    for i in tqdm(range(12)):
        frame = {}
        for j in range(0, 5):
            cell = cell_dims_from_list(list(cells[j, i]))
            cell_image = full_screen_image.crop(cell)
            transcript = pytesseract.image_to_string(cell_image, lang='eng')


            frame[columns[j]] = transcript.replace('\n','').replace('+','').replace('>','').replace('k m', ' km')
        frame['click_target'] = (cells[1, i][0] + 10, cells[1, i][1] + 10) # offset by 10x10 pixels
        frames.append(frame)
    df = pd.DataFrame(frames, columns=columns)
    df.loc[df['Locked'] != '', 'Locked'] = '><'
    return df